# Chains

# 準備

In [2]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv

# 環境変数の読み込み
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.environ["API_KEY"]

# LCEL：Chainの基本的な使い方

In [24]:
from langchain_openai import OpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

model_name = "gpt-3.5-turbo-instruct"

template = "{subject}を勉強する方法をステップ・バイ・ステップで教えてください"

prompt = PromptTemplate.from_template(template)
model = OpenAI(model=model_name)
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [4]:
chain.invoke({"subject": "英語"})

'。\n\nステップ1: 目標を設定する\nまずは、英語を勉強する目的を明確にしましょう。自分が英語をどの程度使えるようになりたいのか、どのようなスキルを身につけたいのかを考え、目標を設定しましょう。\n\nステップ2: 学習方法を選ぶ\n英語を学ぶ方法には、様々なものがあります。自分に合った学習方法を選ぶことで、効率的に英語を身につけることができます。例えば、英会話のスキルを伸ばしたい場合は、会話を重点的に学ぶコースを選ぶことができます。\n\nステップ3: 学習スケジュールを立てる\n'

# Chat Model

In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

model_name = "gpt-3.5-turbo-0125"

template = "あなたは{input_language}から{output_language}に翻訳する優秀な翻訳家です。"
human_template = "{text}"

prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])

model = ChatOpenAI(model=model_name)
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [6]:
chain.invoke({
  "input_language": "英語",
  "output_language": "日本語",
  "text": "I love programming."})

'プログラミングが大好きです。'

# Conversation Chain：会話を続ける方法

In [7]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

model_name = "gpt-3.5-turbo-0125"

model = ChatOpenAI(
    model_name=model_name,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    temperature=1.2,
)

# memoryの使用をLCELで記述する方法が不明なためConversationChainを使う
conversation = ConversationChain(llm=model, memory=ConversationBufferMemory())

In [8]:
while True:
    # ユーザーからの質問を受付
    message = input("メッセージを入力:")

    # 質問が入力されなければ終了
    if message.strip() == "":
        break
    display(f"質問:{message}")

    # 言語モデルに質問
    conversation.invoke(input=message)

print("\n---ご利用ありがとうございました！---")

'質問:안녕하세요?'

안녕하세요! 저는 인공 지능 채팅 로봇입니다. 어떻게 도와 드릴까요?

'質問:인천에서 유명한 일본 라면 가게에 대해 알고 싶어'

인천에서 유명한 일본 라면 가게는 "라멘 무정"이라는 곳이 인기가 많아요. 이 가게는 진짬뽕 라면이 특히 유명한데, 매운 맛을 즐기는 사람들에게 추천해요. 위치는 인천 중구 운중로에 위치해요. 맛집 알려주셔서 감사해요! 다른 정보가 필요하시면 계속 물어봐주세요.

'質問:우동'

우동은 일본의 전통 음식 중 하나로, 쌀가루 반죽을 모양을 내어 기름에 튀긴 것을 국물에 넣은 요리입니다. 대부분 국물을 웃기고, 다시 추가되는 추가재료가 여러가지 있는 돈부리가 주 요리로서食. 추운 날 먹을 때 따뜻하고 푸짐한 맛이 나서 겨울에 많이 섭취됩니다. 일본일뿐만 아니라 동남아시아 국가들에서도 소비되고 있습니다.응원해요! 추가정ísặ니면 묻어주세요!

'質問:우동가게로 소개해줘'

죄송하지만, 제가 알고 있는 정보에는 실제 우동 전문 가게에 대해 구체적으로 기록된 정보가 없습니다. 죄송합니다. 도움이 되는 다른 것이 있으면 말씀해주세요!
---ご利用ありがとうございました！---


# RAGアーキテクチャ（LCEL)

In [3]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

# テキストファイルを読込
# .txt 파일만 로드하도록 필터링
loader = DirectoryLoader(
    path="./data/",
    glob="**/*.txt",  # 서브폴더 포함 모든 .txt
    loader_cls=TextLoader,
    loader_kwargs={"encoding": "utf-8"},  # 필요시 인코딩 지정
)

documents = loader.load()

# チャンクに分割
text_splitter = CharacterTextSplitter(separator="\n", chunk_size=512, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

# Indexの構築
db = Chroma.from_documents(texts, OpenAIEmbeddings())

# 検索（Retriever）の取得
retriever = db.as_retriever()

In [4]:
from langchain_openai import ChatOpenAI

model_name = "gpt-3.5-turbo-0125"

# モデルの作成
model = ChatOpenAI(
    model_name=model_name,
    max_tokens=300,
    temperature=1.2)

In [7]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# プロンプトの作成
prompt = ChatPromptTemplate.from_template("""提供されたコンテキストのみに基づいて次の質問に答えてください:

<コンテキスト>
{context}
</コンテキスト>

Question: {input}""")

# チェーンの作成
chain = (
  {"context": retriever, "input": RunnablePassthrough()}
  | prompt 
  | model 
  | StrOutputParser()
)

In [8]:
# 質問
chain.invoke("キノコストレージについて説明して")

'キノコストレージは容量の大きなデータストレージ装置を指す架空のIT用語であり、その形状がキノコに似ていることから名付けられました。この装置は非常に小さなサイズでありながら、膨大なデータを保持することができるとされています。また、キノコストレージは高い耐久性を持ち、データの保護と安全性を確保すると説明されます。'

# 対話型RetrievalChain

In [9]:
from langchain_community.document_loaders import WikipediaLoader
from langchain_community.document_loaders import PyPDFLoader

# 複数のローダーから読込
loaders = [
    WikipediaLoader(query="SDGs", load_max_docs=3, lang="ja"),
    PyPDFLoader("./data2/20210615_resources_data_guideline_01.pdf")]

documents = []

for loader in loaders:
    documents.extend(loader.load())

documents

[Document(page_content='持続可能な開発目標（じぞくかのうなかいはつもくひょう、英語: Sustainable Development Goals(サステナブル・ディベロップメント・ゴールズ)、略称: SDGs〈エスディージーズ〉）は、2015年9月25日に国連総会で採択された、持続可能な開発のための17の国際目標である。その下に、169の達成基準と232の指標が決められている。\n\n\n== 概要 ==\n2000年9月、アメリカ・ニューヨークでの『国連ミレニアム・サミット』にて採択された国連ミレニアム宣言を基に、「ミレニアム開発目標」（MDGs）が成立した。だがMDGsは2015年までの期限付きであり、2011年、南米コロンビアのフアン・マヌエル・サントス政権にて、期限が近づいても未達成の目標があったことや、それと同時に持続可能な開発をどう進めるかについての議論が提案された。発案はコロンビア外務省経済・社会・環境の女性局長であるパウラ・カバジェーロによるものである。女性外務大臣のマリーア・アンジェラ・オルギンへの稟議を通じて採択された。\n案はグアテマラ等、中南米のいくつかの開発途上国の支援を得たのち、2012年6月にブラジル・リオデジャネイロにて開催された国際サミット『国連持続可能な開発会議』（リオ+20）にて、「2015年以降の次世代MDGsの目標と、持続可能な開発の議論を統一」させることをコロンビア・グアテマラ両国が推奨した。サミットでいかに成果を生み出すかに迷走していたブラジルは、両国の案に乗り、開催されたリオ+20では、その制定への決議を主要な成果として終了した。\n2015年9月25日、国連総会にて、持続可能な開発のために必要不可欠な向こう15年間の新たな行動計画『我々の世界を変革する：持続可能な開発のための2030アジェンダ』（Transforming our world: the 2030 Agenda for Sustainable Development、または単に「2030 Agenda」とも）が採択され、2030年までに達成するべき持続可能な開発目標（SDGs）として、17の世界的目標と169の達成基準が示された。\n\n\n== 17の目標 ==\nSDGsは、以下の17の目標から構成されている。その他、169項

In [10]:
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

# チャンクに分割
text_splitter = CharacterTextSplitter(separator="\n", chunk_size=512, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

# Indexの構築
db = Chroma.from_documents(texts, OpenAIEmbeddings())

# 検索（Retriever）の取得
retriever = db.as_retriever()

In [11]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

model_name = "gpt-3.5-turbo-0125"

# モデルの作成
model = ChatOpenAI(
    model_name=model_name,
    max_tokens=300,
    temperature=1.2)

# Memoryの作成
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# 会話型RetrievalQAチェーンの作成
chain = ConversationalRetrievalChain.from_llm(
    llm=model,
    retriever=retriever,
    max_tokens_limit=4096,
    memory=memory)

In [12]:
# ユーザーからの質問
result = chain.invoke({"question": "SDGsとは何ですか？"})

# 言語モデルからの回答を表示
result["answer"]

'SDGsは「持続可能な開発目標」（Sustainable Development Goals）の略称で、2015年に国連総会で採択された、持続可能な開発を促進するための17の国際目標です。その目標には、貧困や飢餓の撲滅、教育、エネルギー、ジェンダー平等、持続可能な都市開発などが含まれています。'

In [13]:
# ユーザーからの質問
result = chain.invoke({"question": "1つ目の目標について説明してください。出典も示してください"})

# 言語モデルからの回答を表示
result["answer"]

'SDGsの1つ目の目標は、国際連合総会で採択された2015年の「持続可能な開発のための2030アジェンダ」から引用されたもので、目標１は「貧困を無くそう」となっています。'

In [14]:
# ユーザーからの質問
result = chain.invoke({"question": "オープンデータの定義とは？"})

# 言語モデルからの回答を表示
result["answer"]

'オープンデータの定義は以下の要素を含んでいます:\n1. 営利目的、非営利目的を問わず二次利用可能な「ルール」が適用されたもの\n2. 機械判読に適したもの\n3. 無償で利用できるもの'

In [15]:
# 保存されているメッセージを取得
buffer = memory.load_memory_variables({})
print(buffer)

{'chat_history': [HumanMessage(content='SDGsとは何ですか？'), AIMessage(content='SDGsは「持続可能な開発目標」（Sustainable Development Goals）の略称で、2015年に国連総会で採択された、持続可能な開発を促進するための17の国際目標です。その目標には、貧困や飢餓の撲滅、教育、エネルギー、ジェンダー平等、持続可能な都市開発などが含まれています。'), HumanMessage(content='1つ目の目標について説明してください。出典も示してください'), AIMessage(content='SDGsの1つ目の目標は、国際連合総会で採択された2015年の「持続可能な開発のための2030アジェンダ」から引用されたもので、目標１は「貧困を無くそう」となっています。'), HumanMessage(content='オープンデータの定義とは？'), AIMessage(content='オープンデータの定義は以下の要素を含んでいます:\n1. 営利目的、非営利目的を問わず二次利用可能な「ルール」が適用されたもの\n2. 機械判読に適したもの\n3. 無償で利用できるもの')]}


# Summarize Chain：文章の要約

In [16]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter

# PDFの読込
loader = PyPDFLoader("./data2/20210615_resources_data_guideline_01.pdf")
texts = loader.load()

# チャンクに分割
text_splitter = CharacterTextSplitter(separator="\n", chunk_size=512, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [20]:
from langchain_openai import ChatOpenAI

model_name = "chatgpt-4o-latest"

# モデルの作成
model = ChatOpenAI(
    model_name=model_name,
    max_tokens=300,
    temperature=1.2)

In [21]:
from langchain.chains.summarize import load_summarize_chain

summarize_chain = load_summarize_chain(llm=model, chain_type="stuff", verbose=False)
result = summarize_chain.invoke(texts)

In [22]:
result["output_text"]

'持続可能な開発目標（SDGs）は、2030年までに持続可能な社会を実現するための国連による17の目標・169の達成基準からなる国際的枠組みで、2015年に採択された。前身のMDGsの成果と課題を踏まえ、社会・経済・環境の包括的課題に取り組むアジェンダである。進捗は遅れており、特にCOVID-19の影響で貧困対策等に悪影響が出ている。日本でも政府や自治体、企業がSDGs推進を進めており、「SDGs未来都市」などの地域政策や中小企業の参画、有識者・著名人による啓発活動も展開されている。一方、目標達成における課題や市民との齟齬も残る。\n\nまた、「持続可能な開発（Sustainable Development）」の概念は、将来世代の利益を損なわず現代のニーズを満たすことを目的とし、1980年代から環境保全と開発の調和の理念として発展。教育（ESD）や世界遺産政策にも活用されている。\n\nさらに、日本の'

# Sequential Chain

### SimpleSequentialChain

In [26]:
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMCheckerChain, SimpleSequentialChain

llm = OpenAI(temperature=0.7)

# 1つ目のChain
template = "{subject}に関する問題を日本語で作成してください。答えは作成しないでください"
llm_chain = LLMChain(
    llm=model,
    prompt=PromptTemplate.from_template(template),
)

# 2つ目のChain
check_chain = LLMCheckerChain.from_llm(
    llm=llm)

# Chain
overall_chain = SimpleSequentialChain(
    chains=[llm_chain, check_chain],
    verbose=True)

In [27]:
# Simple Sequential Chainの実行
overall_chain.invoke("動物")



> Entering new SimpleSequentialChain chain...


「日本の動物園で問題となっている、動物のストレスや健康管理についての課題は何ですか？」


> Entering new SequentialChain chain...

> Finished chain.
 日本の動物園で問題となっている、動物のストレスや健康管理についての課題は、動物の自然環境からの離脱や、過密な展示空間、適切な運動や栄養管理の欠如、繁殖の過剰などが挙げられます。これらの課題を解決するためには、動物園の展示や管理方法の見直し、専門的な動物の健康管理や行動学に基づいたケアの充実が必要です。また、観光客や来園者への啓発や教育も重要です。

> Finished chain.


{'input': '動物',
 'output': ' 日本の動物園で問題となっている、動物のストレスや健康管理についての課題は、動物の自然環境からの離脱や、過密な展示空間、適切な運動や栄養管理の欠如、繁殖の過剰などが挙げられます。これらの課題を解決するためには、動物園の展示や管理方法の見直し、専門的な動物の健康管理や行動学に基づいたケアの充実が必要です。また、観光客や来園者への啓発や教育も重要です。'}

### SequentialChain

In [28]:
from langchain_core.prompts import PromptTemplate
from langchain.chains import SequentialChain

# 1つ目のChain
template1 = "{theme}に関する{target}向けのブログ記事のタイトルを考えてください。"
llm_chain1 = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(template1),
    output_key="title"
)

# 2つ目のChain
template2 = "ブログ記事「{title}」の見出しを考えてください。"
llm_chain2 = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(template2),
    output_key="headline"
)

# Chain
overall_chain = SequentialChain(
    chains=[llm_chain1, llm_chain2],
    input_variables=["theme", "target"],
    output_variables=["title", "headline"],
    verbose=True)

In [29]:
# Chainの実行
overall_chain.invoke({
    "theme":"観光",
    "target": "日本の富裕層"})



> Entering new SequentialChain chain...

> Finished chain.


{'theme': '観光',
 'target': '日本の富裕層',
 'title': '\n\n1. 「贅沢な旅の楽しみ方：日本の絶景スポットを巡る旅」\n2. 「VIPな旅のススメ：日本の高級ホテルとリゾートを満喫」\n3. 「究極のグルメ体験：日本の名店を堪能する旅」\n4. 「大人の隠れ家：日本の秘境を訪ねる旅」\n5. 「伝統とモダンの融合：日本の美しい文化を巡る旅」\n6. 「贅を尽くした旅のはじまり：日本の豪華なトランスポートでの旅」\n7. 「自然との調和：日本のリゾートでゆったりとした休暇を」\n8. 「日本の歴史を感じる旅：古都を巡る文',
 'headline': '\n9. 「美しい四季の旅：日本の色鮮やかな自然を楽しむ」\n10. 「芸術との邂逅：日本の美術館や博物館を訪ねる旅」\n11. 「心を癒す旅：日本の温泉地でのリラックスタイム」\n12. 「夢のような体験：日本のテーマパークを巡る旅」\n13. 「宿場町巡りの旅：日本の歴史的な街並みを散策する」\n14. 「和の心を感じる旅：茶道や着物体験などの文化体験」\n15. 「大自然の驚異：日本の絶景を眺めるドライブ旅」\n16. 「日本の島々への旅：'}